In [1]:
import numpy as np
from pathlib import Path
from PIL import Image, ImageFilter, ImageEnhance

In [2]:
def low_light_simple(img,
                     brightness=0.15,     # reduce exposure ~45%
                     contrast=0.85,       # slightly softer contrast
                     noise_std=0.015,     # small, realistic noise
                     blur_radius=0.2):    # low-light softness
                     
    img = img.convert("RGB")

    # ----- 1) Reduce brightness -----
    img = ImageEnhance.Brightness(img).enhance(brightness)

    # ----- 2) Reduce contrast -----
    img = ImageEnhance.Contrast(img).enhance(contrast)

    # Convert to array for noise
    arr = np.asarray(img).astype(np.float32) / 255.0

    # ----- 3) Add slight Gaussian noise -----
    noise = np.random.normal(0, noise_std, arr.shape).astype(np.float32)
    arr = np.clip(arr + noise, 0.0, 1.0)

    img = Image.fromarray((arr * 255).astype(np.uint8))

    # ----- 4) Mild blur to match smartphone low-light softness -----
    img = img.filter(ImageFilter.GaussianBlur(blur_radius))

    return img

In [3]:
input_path = Path('images')
output_path = Path('processed')

for I in input_path.iterdir():
    filename = I.name
    Img = Image.open(I)
    low_I = low_light_simple(img=Img)
    low_I.save(output_path/filename, quality=95)
